In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install -y build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install scikit-surprise --no-cache-dir

In [ ]:
!pip install "numpy<2"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pickle
from scipy.sparse import csr_matrix, save_npz, load_npz

In [ ]:
# Xu ly file dataset ve format users-movies
rating_df = pd.read_csv('/content/drive/MyDrive/Project_AIL303_GROUP2/Final_dataset_in_here!!!!!/100k_final_dataset.csv')[:30000]
train_df, test_df = train_test_split(rating_df, test_size=0.2, random_state=42)
users_movies_matrix =train_df.pivot_table(index='userId', columns='movieId', values='rating')
users_movies_matrix.fillna(0, inplace = True)
users_movies_matrix.head(10)

movieId,1,2,3,6,7,10,11,12,15,16,...,289651,289695,289737,289985,290407,290515,290521,290573,291258,292033
userId,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dtest = (users_movies_matrix>0).sum(axis = 1)
print(dtest)

userId
22         2
34         2
55         1
58         5
81         4
        ... 
19315      2
19347      1
19358      1
19366      2
19387    100
Length: 1531, dtype: int64


In [ ]:
def normalize_users_movies_matrix(ty='mean', user_movie_matrix= None):
  if ty == 'mean':
    return user_movie_matrix.apply(lambda value: value - value.mean(), axis=1)
  elif ty == 'median':
    return user_movie_matrix.apply(lambda value: value - value.median(), axis=1)
  else:
    raise ValueError('Not support this normalization type.')


In [ ]:
users_movies_matrix = normalize_users_movies_matrix(ty='mean', user_movie_matrix=users_movies_matrix)
users_movies_matrix.head()

movieId,1,2,3,6,7,10,11,12,15,16,...,289651,289695,289737,289985,290407,290515,290521,290573,291258,292033
userId,,,,,,,,,,,,,,,,,,,,,
22,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,...,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730,-0.000730
34,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,...,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899,-0.000899
55,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,...,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449
58,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,...,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909,-0.001909
81,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,...,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134,-0.002134


In [ ]:
users_sim_matrix = cosine_similarity(users_movies_matrix)
users_sim_df = pd.DataFrame(users_sim_matrix,
                                index = users_movies_matrix.index,
                                columns=users_movies_matrix.index)
users_sim_df.head()
user_means = train_df.groupby('userId')['rating'].mean()

In [ ]:
def get_sim_vecter_of_new_user(new_user_ratings, users_movies_matrix):
    '''
    new_user_rating : dict
    users_movies_matrix: pd.DataFrame
    '''
    new_user_vector_full = pd.Series(new_user_ratings).reindex(users_movies_matrix.columns, fill_value=0)
    return cosine_similarity(new_user_vector_full.values.reshape(1, -1), users_movies_matrix)

def get_similar_vector(user_sim_df, user_movie_matrix = None, userId= None, new_user_rating = None):
    if userId is not None:
      if userId not in user_sim_df.index:
         raise ValueError(f"User ID {userId} không tồn tại trong sim_matrix.")
      sim_vector = user_sim_df.loc[userId]
      return sim_vector.drop(userId, errors= 'ignore')

    elif new_user_rating is not None:
      sim_vector_array = get_sim_vecter_of_new_user(new_user_rating, user_movie_matrix)
      return pd.Series(sim_vector_array[0], index=user_movie_matrix.index)
    else:
      raise ValueError("Phải cung cấp user_id hoặc new_user_ratings.")

def predict_rating_flexible(movieid_to_predict, k,
                            users_movies_matrix,
                            sim_vector
                            ):

    top_k_neighbors = sim_vector.nlargest(k)
    if movieid_to_predict not in users_movies_matrix.columns:
        return users_movies_matrix.values[np.where(users_movies_matrix.values> 0)].mean()
    neighbor_ratings_about_movieId = users_movies_matrix.loc[top_k_neighbors.index, movieid_to_predict]
    valid_neighbors = neighbor_ratings_about_movieId[neighbor_ratings_about_movieId > 0]

    if valid_neighbors.empty:
        return users_movies_matrix.values[np.where(users_movies_matrix.values > 0)].mean()

    valid_neighbors_sims = top_k_neighbors.loc[valid_neighbors.index]
    predicted_rating = np.dot(valid_neighbors_sims, valid_neighbors) / valid_neighbors_sims.sum()
    return predicted_rating

def set_available_userId(users_movies_matrix):
  return set(users_movies_matrix.index)

def set_available_movieId(users_movies_matrix):
  return set(users_movies_matrix.columns)

def check_old_userId(userId, avail_userId):
  return userId in avail_userId

def check_old_movieId(movieId, avail_movieId):
  return movieId in avail_movieId

def cal_rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))

def eval(test_df, users_movies_matrix, users_sim_df, k, users_mean):
  pred = []
  gt_ratings = []
  avail_userId = set_available_userId(users_movies_matrix)
  for _, row in test_df.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    gt_rate = row['rating']
    if check_old_userId(user_id, avail_userId):
      sim_vector = get_similar_vector(users_sim_df, userId=user_id)
      # user_means_rating = user_means.loc[user_id]
      pred_rate = predict_rating_flexible(
            movie_id,
            k,
            users_movies_matrix,
            sim_vector
        )
      # pred_rate =np.clip(pred_rate + user_means_rating, 0.5, 5)

      pred.append(pred_rate)
      gt_ratings.append(gt_rate)

  return cal_rmse(y_test=gt_ratings, y_pred=pred)

In [ ]:
rs_rmse_on_testdf = eval(test_df, users_movies_matrix, users_sim_df, k=5,users_mean= user_means)
print(f"RMSE on test dataset: {rs_rmse_on_testdf}")

RMSE on test dataset: 1.1422990120273757


4. Create sim_matrix and userId_movieId_matrix on 100k of Dataset

In [ ]:
df_100k = pd.read_csv('/content/drive/MyDrive/Project_AIL303_GROUP2/Final_dataset_in_here!!!!!/100k_final_dataset.csv')[:30000]
df_100k.to_csv('/content/drive/MyDrive/Project_AIL303_GROUP2/Final_dataset_in_here!!!!!/final_dataset.csv', index=False)
userId_movieId_matrix = df_100k.pivot_table(index='userId', columns='movieId', values='rating')
userId_movieId_matrix.fillna(0, inplace = True)
sim_matrix = cosine_similarity(userId_movieId_matrix)
users_sim_df = pd.DataFrame(sim_matrix,
                                index = userId_movieId_matrix.index,
                                columns=userId_movieId_matrix.index)

with open('/content/drive/MyDrive/Project_AIL303_GROUP2/Final_dataset_in_here!!!!!/userId_movieId_matrix.pkl', 'wb') as f:
    pickle.dump(userId_movieId_matrix, f)
# with open('/content/drive/MyDrive/Project_AIL303_GROUP2/Final_dataset_in_here!!!!!/userId_similarity.pkl', 'wb') as f:
#     pickle.dump(users_sim_df, f)

print(userId_movieId_matrix)
users_sim_df.head()

movieId  1       2       3       6       7       9       10      11      \
userId                                                                    
22          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
34          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
55          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
58          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
81          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
19347       0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19355       0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19358       0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19366       0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19387       0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  12      13     

userId,22,34,55,58,81,101,109,132,144,174,...,19247,19264,19283,19314,19315,19347,19355,19358,19366,19387
userId,,,,,,,,,,,,,,,,,,,,,
22,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0.0,0.0,0.0,0.0,1.0,0.0,0.084065,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(type(users_sim_df))
print(type(userId_movieId_matrix))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
with open('/content/drive/MyDrive/Project_AIL303_GROUP2/Final_dataset_in_here!!!!!/userId_movieId_matrix.pkl', 'rb') as f:
    dt=pickle.load(f)
dt = pd.DataFrame(dt)
dt.head()
print((dt>0).sum(axis=1))


userId
22         2
34         2
55         1
58         7
81         4
        ... 
19347      1
19355      1
19358      1
19366      2
19387    123
Length: 1675, dtype: int64
